In [ ]:
import numpy as np
import pandas as pd
import requests
import ast

In [ ]:
results = []
offset = 0
NUM_CLUES = 156809

try:
    r = requests.get('http://jservice.io/api/clues?offset=%d' % offset)
except requests.exceptions.HTTPError as e:
    print("error: %s" %(e))
    
while offset < NUM_CLUES:
    try:
        r = requests.get('http://jservice.io/api/clues?offset=%d' % offset)
    except requests.exceptions.HTTPError as err:
        print(err)
    if (offset % 10000) == 0:
        print("done %d" % offset)

In [ ]:
raw_data = pd.DataFrame.from_dict(results[0].json())

for i in range(1, 1568):
    raw_data = raw_data.append(results[i].json())
    if (i % 100) == 0:
        print('done %d' % i)

In [ ]:
raw_data.head()

In [ ]:
raw_data.info()

In [ ]:
raw_data.to_csv('raw_api_data')

In [ ]:
new_data = raw_data.drop(['game_id'], 1)
new_data.drop(['invalid_count'], 1, inplace=True)
new_data = new_data[pd.notnull(new_data['value'])]
new_data = new_data[pd.notnull(new_data['category_id'])]
new_data = new_data[pd.notnull(new_data['category'])]

new_data = new_data[new_data.answer != '=']
new_data = new_data[new_data.question != '[filler]']
new_data = new_data[new_data.question != '[video clue]']
new_data = new_data[new_data.question != '[audio clue]']
new_data = new_data[new_data.question != '(audio clue)']
new_data = new_data[new_data.question != '(audio clue)']

In [ ]:
new_data.to_csv('cleaned_api_data')

In [ ]:
new_data.head()

In [ ]:
new_data.info()

In [ ]:
raw_cat_data = pd.DataFrame.from_dict(new_data.loc[:,'category'][:])



In [ ]:
raw_cat_data.head()

In [ ]:
split_cat_data = pd.DataFrame()

i = 0

for row in raw_cat_data.iterrows():
    try:
        cat_info = str(split_cat_data.iloc[i, 0])
        cat_breakdown = ast.literal_eval(cat_info)
        split_cat_data = split_cat_data.append(cat_breakdown, ignore_index=True)
    except ValueError as e:
        print('error was %s at %d: %s' %(e, i, cat_info)
    
    i += 1
    if (i % 10000) == 0:
              print('done %d' %i)        

In [ ]:
split_cat_data.head()

In [ ]:
new_data['category_name'] = pd.Series(split_cat_data['title'])

In [ ]:
new_data.head()

In [ ]:
new_data.to_csv('final_api_data')